In [213]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from aif360.algorithms.inprocessing import GerryFairClassifier
from sklearn.svm import SVC
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import sklearn

import aif360
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions\
            import get_distortion_adult, get_distortion_german, get_distortion_compas

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas

from aif360.datasets import AdultDataset, BankDataset, CompasDataset, GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms import preprocessing, inprocessing, postprocessing
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.lfr import LFR
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
import copy

from IPython.display import Markdown, display
import warnings
warnings.filterwarnings('ignore')
%load_ext jupyternotify
np.random.seed(1)

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [214]:
def run_classification_metrics(CM:ClassificationMetric):
    def f1_score(priv=None):
            numer = CM.num_true_positives(privileged=priv)
            denom = CM.num_true_positives(privileged=priv) + 0.5*float(CM.num_false_positives(privileged=priv) + CM.num_false_negatives(privileged=priv))
            return float(numer/denom)
    return np.array([
        round(CM.accuracy(), 4),
        round(f1_score(), 4),
        round(CM.theil_index(), 4),
        round(CM.false_positive_rate(privileged=False), 4),
        round(CM.false_positive_rate(privileged=True), 4),
        round(CM.false_negative_rate(privileged=False), 4),
        round(CM.false_negative_rate(privileged=True), 4),
        round(1-CM.error_rate(privileged=False), 4),
        round(1-CM.error_rate(privileged=True), 4),
        round(CM.false_discovery_rate(privileged=False), 4),
        round(CM.false_discovery_rate(privileged=True), 4),
        round(CM.false_omission_rate(privileged=False), 4),
        round(CM.false_omission_rate(privileged=True), 4),
        
        #all results
        CM.num_true_positives(),
        CM.num_true_negatives(),
        CM.num_false_positives(),
        CM.num_false_negatives(),
        
        #privileged
        CM.num_true_positives(privileged=True),
        CM.num_true_negatives(privileged=True),
        CM.num_false_positives(privileged=True),
        CM.num_false_negatives(privileged=True),
        
        #unprivileged
        CM.num_true_positives(privileged=False),
        CM.num_true_negatives(privileged=False),
        CM.num_false_positives(privileged=False),
        CM.num_false_negatives(privileged=False),
        
        round(f1_score(True), 4),
        round(f1_score(False), 4),
    ])

In [215]:
def run_binary_dataset_metrics(BLDM:BinaryLabelDatasetMetric):
    #print("Consistency: ", BLDM.consistency())
    return np.array([
        round(BLDM.base_rate(privileged=True), 4), # 1 means privileged bias
        round(BLDM.base_rate(privileged=False), 4), # 1 means unprivileged bias
        round(BLDM.consistency()[0], 4)
    ])

In [216]:
def get_model_name(model):
    if isinstance(model, sklearn.linear_model.LogisticRegression):
        return "Logistic Regression"
    if isinstance(model, sklearn.linear_model.LinearRegression):
        return "Linear Regression"
    if isinstance(model, sklearn.ensemble.BaggingClassifier):
        return "Meta Classifier"
    
    if isinstance(model, preprocessing.DisparateImpactRemover):
        return "DIR"
    if isinstance(model, preprocessing.LFR):
        return "LFR"
    if isinstance(model, preprocessing.OptimPreproc):
        return "OP"
    if isinstance(model, preprocessing.Reweighing):
        return "RW"
    
    if isinstance(model, inprocessing.PrejudiceRemover):
        return "PR"
    if isinstance(model, inprocessing.AdversarialDebiasing):
        return "AD"
    if isinstance(model, inprocessing.ARTClassifier):
        return "ARTC"
    if isinstance(model, inprocessing.ExponentiatedGradientReduction):
        return "EGR"
    if isinstance(model, inprocessing.GerryFairClassifier):
        return "GFC"
    if isinstance(model, inprocessing.GridSearchReduction):
        return "GSR"
    if isinstance(model, inprocessing.MetaFairClassifier):
        return "MFC"
    
    if isinstance(model, postprocessing.EqOddsPostprocessing):
        return "EOP"
    if isinstance(model, postprocessing.CalibratedEqOddsPostprocessing):
        return "CEOP"
    if isinstance(model, postprocessing.RejectOptionClassification):
        return "ROC"
    
    return "None"

In [217]:
def get_dataset_name(dataset):
    if isinstance(dataset, aif360.datasets.german_dataset.GermanDataset):
        return "German Dataset"
    if isinstance(dataset, aif360.datasets.adult_dataset.AdultDataset):
        return "Adult Dataset"
    if isinstance(dataset, aif360.datasets.bank_dataset.BankDataset):
        return "Bank Dataset"
    if isinstance(dataset, aif360.datasets.compas_dataset.CompasDataset):
        return "Compas Dataset"

In [296]:
# get specialized dataset for Optimized Preprocessing
def get_OP_dataset(dataset_name):
    dataset = None
    if dataset_name=="bank":
        return
    if dataset_name=="adult":
        dataset = load_preproc_data_adult(['sex'])
    elif dataset_name=="compas":
        dataset = load_preproc_data_compas(['race'])
    elif dataset_name=="german":
        dataset = load_preproc_data_german(['age'])
    np.random.seed(0)
    dataset_train, dataset_test = dataset.split([0.7], shuffle = True)
    return dataset_train, dataset_test

In [297]:
def get_dataset_options(dataset_name):
    optim_options = None
    if dataset_name=="adult":
        optim_options = {
        "distortion_fun": get_distortion_adult,
        "epsilon": 0.05,
        "clist": [0.99, 1.99, 2.99],
        "dlist": [.1, 0.05, 0]
        }
        pro_attr = 'sex'
        return (AdultDataset(), pro_attr, [{'sex': 1}], [{'sex': 0}], optim_options)
    elif dataset_name=="compas":
        optim_options = {
        "distortion_fun": get_distortion_compas,
        "epsilon": 0.05,
        "clist": [0.99, 1.99, 2.99],
        "dlist": [.1, 0.05, 0]
        }
        pro_attr = 'race'
        return (CompasDataset(), pro_attr, [{'race': 1}], [{'race': 0}], optim_options)
    elif dataset_name=="bank":
        pro_attr = 'age'
        return (BankDataset(protected_attribute_names=['age'],
            privileged_classes=[lambda x: x >= 25], 
            features_to_drop=['day_of_week']), pro_attr, [{'age': 1}], [{'age': 0}], None)
    elif dataset_name=="german":
        optim_options = {
            "distortion_fun": get_distortion_german,
            "epsilon": 0.1,
            "clist": [0.99, 1.99, 2.99],
            "dlist": [.1, 0.05, 0]
        }   
        pro_attr = 'age'
        #g = load_preproc_data_german(['age'])
        #label_map = {1.0: 'Good Credit', 0.0: 'Bad Credit'}
        #g = GermanDataset(metadata={'label_map': label_map})
        g = GermanDataset()
        #g.labels = g.labels-1
        # load_preproc_data_german(['age'])
        return (g, pro_attr, 
                    [{'age': 1}], [{'age': 0}], optim_options)

In [307]:
def analyze_algo(dataset_train, dataset_test, privileged_groups, unprivileged_groups, classifier=None, 
                 preprocessing_algo=None, inprocessing_algo=None, postprocessing_algo=None):
    print(get_model_name(preprocessing_algo), get_model_name(inprocessing_algo), get_model_name(postprocessing_algo))
    model = sklearn.linear_model.LogisticRegression() # solver='liblinear', class_weight='balanced', 
    
    dataset_train_pred = dataset_train.copy(deepcopy=True)
    dataset_test_pred = dataset_test.copy(deepcopy=True)
            
    if preprocessing_algo is not None:
        dataset_train_pred = preprocessing_algo.fit_transform(dataset_train_pred)
        dataset_train_pred = dataset_train.align_datasets(dataset_train_pred)
    
    if inprocessing_algo is not None:
        inp = inprocessing_algo
        inp.fit(dataset_train_pred)
        dataset_train_pred = inp.predict(dataset_train_pred)
        dataset_test_pred = inp.predict(dataset_test_pred) 
    else:
        model.fit(dataset_train_pred.features, dataset_train_pred.labels)   # .ravel()
        fav_idx = np.where(model.classes_ == dataset_train.favorable_label)[0][0]
        dataset_train_pred.scores = model.predict_proba(dataset_train_pred.features)[:,fav_idx].reshape(-1,1) 
        dataset_train_pred.labels = model.predict(dataset_train_pred.features).reshape(-1,1) 
        dataset_test_pred.scores = model.predict_proba(dataset_test_pred.features)[:,fav_idx].reshape(-1,1) 
        dataset_test_pred.labels = model.predict(dataset_test_pred.features).reshape(-1,1)  
            
    if postprocessing_algo is not None:
        pp = postprocessing_algo
        pp = pp.fit(dataset_train, dataset_train_pred)
        dataset_test_pred = pp.predict(dataset_test_pred)

    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    
    BLDM = BinaryLabelDatasetMetric(dataset_test_pred,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    name = ""
    if preprocessing_algo is not None:
        name += get_model_name(preprocessing_algo) + " + "
    if inprocessing_algo is not None:
        name += get_model_name(inprocessing_algo) + " + "
    if postprocessing_algo is not None:
        name += get_model_name(postprocessing_algo)
    if name == "":
        name = get_model_name(model)
        
    if name.endswith(" + "):
        lastIndex = name.rindex(" + ")
        name = name[:lastIndex]
    
    #print(run_classification_metrics(CM))
    #print(run_binary_dataset_metrics(BLDM))
    metrics = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))
       
    return {"key":name, "val":metrics}

In [312]:
datasets = ["compas", "german", "bank", "adult"]
#datasets = ["compas"]
for dataset_name in datasets:
    dataset, pro_attr, privileged_groups, unprivileged_groups, optim_options = get_dataset_options(dataset_name)
    
    print("DATASET NAME: ", dataset_name)
    np.random.seed(0)
    dataset_train, dataset_test = dataset.split([0.7], shuffle = True)

    preprocessing_algos = [None,
    #                       LFR(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, k=10, Ax=0.1, Ay=1.0, Az=2.0, verbose=0),
    #                      OptimPreproc(OptTools, optim_options, unprivileged_groups = unprivileged_groups, privileged_groups = privileged_groups),
                          #preprocessing.Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups),
    #                      preprocessing.DisparateImpactRemover(sensitive_attribute=pro_attr),
                           ]
                          
    inprocessing_algos = [None,
                           #inprocessing.AdversarialDebiasing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=0),
                          GerryFairClassifier(),
                          inprocessing.PrejudiceRemover(sensitive_attr=pro_attr),
                          inprocessing.ExponentiatedGradientReduction(sklearn.linear_model.LogisticRegression(), constraints="DemographicParity", drop_prot_attr=False),
                          inprocessing.GridSearchReduction(sklearn.linear_model.LogisticRegression(), prot_attr=pro_attr, constraints="DemographicParity", drop_prot_attr=False),
                          ]

    postprocessing_algos = [None,
    #                        postprocessing.EqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=0),
    #                        postprocessing.CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=0),
    #                        postprocessing.RejectOptionClassification(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups),  
                            ]

    df = {}

    for pre in preprocessing_algos:
        for inproc in inprocessing_algos:
            for post in postprocessing_algos:    
                
                # get specialized dataset for Optimized Preprocessing technique
                if get_model_name(pre)=="OP" and dataset_name!="bank":
                    print("Specialized function: ", dataset_name)
                    dataset_train, dataset_test = get_OP_dataset(dataset_name) 
                
                try:
                    res = analyze_algo(dataset_train, dataset_test, privileged_groups, unprivileged_groups, 
                             preprocessing_algo=copy.deepcopy(pre), 
                             inprocessing_algo=copy.deepcopy(inproc),
                             postprocessing_algo=copy.deepcopy(post))
                    df[res["key"]] = res["val"]

                except KeyboardInterrupt:
                    raise KeyboardInterrupt()
                except Exception as e:
                    print("FAILED: " + get_model_name(pre) + ", " + get_model_name(inproc) + ", " + get_model_name(post) + " on dataset " + get_dataset_name(dataset), e)

    df = pd.DataFrame.from_dict(df)                
    df.index = ["Accuracy", "F1 Score", "Theil Index",
                    "False Positive Rate - Unprivileged", "False Positive Rate - Privileged",
                    "False Negative Rate - Unprivileged", "False Negative Rate - Privileged",
                    "Accuracy - Unprivileged", "Accuracy - Privileged",
                    "False Discovery Rate - Unprivileged", "False Discovery Rate - Privileged",
                    "False Omission Rate - Unprivileged", "False Omission Rate - Privileged",
                    "Num True Pos", "Num True Neg", "Num False Pos", "Num False Neg",
                    "Num True Pos - Privileged", "Num True Neg - Privileged", "Num False Pos - Privileged", "Num False Neg - Privileged",
                    "Num True Pos - Unprivileged", "Num True Neg - Unprivileged", "Num False Pos - Unprivileged", "Num False Neg - Unprivileged",
                    "F1 Score - Privileged", "F1 Score - Unprivileged",
                    "Privileged base Rate", "Unprivileged base Rate", "Consistency"]

    df = df.T
    df.to_csv("Data -"+ dataset_name +".csv", sep=',', encoding='utf-8')
    display(df)        

DATASET NAME:  compas
None None None
None GFC None
None PR None
None EGR None
None GSR None


,Accuracy,F1 Score,Theil Index,False Positive Rate - Unprivileged,False Positive Rate - Privileged,False Negative Rate - Unprivileged,False Negative Rate - Privileged,Accuracy - Unprivileged,Accuracy - Privileged,False Discovery Rate - Unprivileged,...,Num False Neg - Privileged,Num True Pos - Unprivileged,Num True Neg - Unprivileged,Num False Pos - Unprivileged,Num False Neg - Unprivileged,F1 Score - Privileged,F1 Score - Unprivileged,Privileged base Rate,Unprivileged base Rate,Consistency
Logistic Regression,0.6953,0.7377,0.1766,0.3470,0.5783,0.2596,0.1491,0.6972,0.6914,0.3135,...,58.0,462.0,397.0,211.0,162.0,0.7761,0.7124,0.7496,0.5463,0.8903
GFC,0.6650,0.7451,0.1168,0.5641,0.7391,0.1298,0.0668,0.6558,0.6834,0.3871,...,26.0,543.0,265.0,343.0,81.0,0.7874,0.7192,0.8611,0.7192,0.8916
PR,0.3079,0.2557,0.6176,0.6628,0.3696,0.7212,0.8817,0.3076,0.3086,0.6984,...,343.0,174.0,205.0,403.0,450.0,0.1769,0.2898,0.2116,0.4683,0.8859
EGR,0.6786,0.7200,0.1946,0.3931,0.4696,0.2500,0.2365,0.6794,0.6769,0.3380,...,92.0,468.0,369.0,239.0,156.0,0.7481,0.7032,0.6543,0.5739,0.8492
GSR,0.6953,0.7377,0.1766,0.3470,0.5783,0.2596,0.1491,0.6972,0.6914,0.3135,...,58.0,462.0,397.0,211.0,162.0,0.7761,0.7124,0.7496,0.5463,0.8903


DATASET NAME:  german
None None None
None GFC None
FAILED: None, GFC, None on dataset German Dataset at least one array or dtype is required
None PR None
None EGR None
None GSR None
FAILED: None, GSR, None on dataset German Dataset Supplied y labels are not 0 or 1


,Accuracy,F1 Score,Theil Index,False Positive Rate - Unprivileged,False Positive Rate - Privileged,False Negative Rate - Unprivileged,False Negative Rate - Privileged,Accuracy - Unprivileged,Accuracy - Privileged,False Discovery Rate - Unprivileged,...,Num False Neg - Privileged,Num True Pos - Unprivileged,Num True Neg - Unprivileged,Num False Pos - Unprivileged,Num False Neg - Unprivileged,F1 Score - Privileged,F1 Score - Unprivileged,Privileged base Rate,Unprivileged base Rate,Consistency
Logistic Regression,0.7333,0.8261,0.1195,0.5385,0.6883,0.3448,0.0552,0.5952,0.7558,0.2692,...,10.0,19.0,6.0,7.0,10.0,0.8444,0.6909,0.8682,0.6190,0.7927
PR,0.6267,0.8664,0.1260,0.6154,0.6494,0.0000,0.0000,0.4048,0.6628,0.3200,...,0.0,17.0,0.0,8.0,0.0,0.8724,0.8095,0.8566,0.5952,0.7733
EGR,0.7400,0.8282,0.1251,0.8462,0.5844,0.1034,0.1050,0.6667,0.7519,0.2973,...,19.0,26.0,2.0,11.0,3.0,0.8351,0.7879,0.8023,0.8810,0.7680


DATASET NAME:  bank
None None None
None GFC None
None PR None
None EGR None
None GSR None


,Accuracy,F1 Score,Theil Index,False Positive Rate - Unprivileged,False Positive Rate - Privileged,False Negative Rate - Unprivileged,False Negative Rate - Privileged,Accuracy - Unprivileged,Accuracy - Privileged,False Discovery Rate - Unprivileged,...,Num False Neg - Privileged,Num True Pos - Unprivileged,Num True Neg - Unprivileged,Num False Pos - Unprivileged,Num False Neg - Unprivileged,F1 Score - Privileged,F1 Score - Unprivileged,Privileged base Rate,Unprivileged base Rate,Consistency
Logistic Regression,0.8933,0.5020,0.0912,0.0604,0.0342,0.6818,0.5813,0.7742,0.8966,0.3438,...,654.0,21.0,171.0,11.0,45.0,0.5059,0.4286,0.0828,0.1290,0.9825
GFC,0.8909,0.4406,0.1000,0.0549,0.0244,0.6818,0.6693,0.7782,0.8940,0.3226,...,753.0,21.0,172.0,10.0,45.0,0.4410,0.4330,0.0632,0.1250,0.9885
PR,0.8940,0.4995,0.0917,0.0604,0.0324,0.6515,0.5902,0.7823,0.8971,0.3235,...,664.0,23.0,171.0,11.0,43.0,0.5016,0.4600,0.0801,0.1371,0.9907
EGR,0.8938,0.5074,0.0904,0.0330,0.0352,0.7273,0.5716,0.7823,0.8970,0.2500,...,643.0,18.0,176.0,6.0,48.0,0.5125,0.4000,0.0850,0.0968,0.9781
GSR,0.8934,0.4946,0.0924,0.0330,0.0328,0.7727,0.5893,0.7702,0.8968,0.2857,...,663.0,15.0,176.0,6.0,51.0,0.5016,0.3448,0.0806,0.0847,0.9788


DATASET NAME:  adult
None None None
None GFC None
None PR None
None EGR None
None GSR None


,Accuracy,F1 Score,Theil Index,False Positive Rate - Unprivileged,False Positive Rate - Privileged,False Negative Rate - Unprivileged,False Negative Rate - Privileged,Accuracy - Unprivileged,Accuracy - Privileged,False Discovery Rate - Unprivileged,...,Num False Neg - Privileged,Num True Pos - Unprivileged,Num True Neg - Unprivileged,Num False Pos - Unprivileged,Num False Neg - Unprivileged,F1 Score - Privileged,F1 Score - Unprivileged,Privileged base Rate,Unprivileged base Rate,Consistency
Logistic Regression,0.8241,0.5945,0.1461,0.0077,0.1225,0.7103,0.4335,0.9131,0.7813,0.1724,...,1228.0,144.0,3881.0,30.0,353.0,0.6158,0.4292,0.2599,0.0395,0.9371
GFC,0.2454,0.3942,0.0340,1.0000,1.0000,0.0000,0.0000,0.1127,0.3093,0.8873,...,0.0,497.0,0.0,3911.0,0.0,0.4725,0.2027,1.0000,1.0000,1.0000
PR,0.8368,0.6342,0.1318,0.0207,0.1141,0.5151,0.4077,0.9235,0.7951,0.2516,...,1155.0,241.0,3830.0,81.0,256.0,0.6413,0.5885,0.2620,0.0730,0.9328
EGR,0.8231,0.5760,0.1538,0.1051,0.0457,0.2495,0.5563,0.8786,0.7964,0.5242,...,1576.0,373.0,3500.0,411.0,124.0,0.5741,0.5824,0.1688,0.1779,0.9151
GSR,0.8241,0.5945,0.1461,0.0077,0.1225,0.7103,0.4335,0.9131,0.7813,0.1724,...,1228.0,144.0,3881.0,30.0,353.0,0.6158,0.4292,0.2599,0.0395,0.9371


In [299]:
dataset_name = "compas"
dataset, pro_attr, privileged_groups, unprivileged_groups, optim_options = get_dataset_options(dataset_name)

#dataset = AdultDataset()
#label_map = {1.0: 'Good Credit', 0.0: 'Bad Credit'}
#dataset = GermanDataset(metadata={'label_map': label_map})
#dataset.labels = dataset.labels-1

np.random.seed(0)
dataset_train, dataset_test = dataset.split([0.7], shuffle = True)

print("Specialized function: ", dataset_name)
dataset_train, dataset_test = get_OP_dataset(dataset_name)

dataset_train_pred = dataset_train.copy(deepcopy=True)
dataset_test_pred = dataset_test.copy(deepcopy=True)

#inp = preprocessing.DisparateImpactRemover(sensitive_attribute=pro_attr)
inp = OptimPreproc(OptTools, optim_options, unprivileged_groups = unprivileged_groups, privileged_groups = privileged_groups)
dataset_train_pred = inp.fit_transform(dataset_train_pred)

X_train = dataset_train_pred.features
y_train = dataset_train_pred.labels #.ravel()

model = sklearn.linear_model.LogisticRegression() # solver='liblinear', class_weight='balanced', 
model.fit(X_train, y_train)

fav_idx = np.where(model.classes_ == dataset_train.favorable_label)[0][0]
dataset_train_pred.scores = model.predict_proba(dataset_train_pred.features)[:,fav_idx].reshape(-1,1) 
dataset_train_pred.labels = model.predict(dataset_train_pred.features).reshape(-1,1)
dataset_test_pred.scores = model.predict_proba(dataset_test_pred.features)[:,fav_idx].reshape(-1,1) 
dataset_test_pred.labels = model.predict(dataset_test_pred.features).reshape(-1,1)     

#inp = inprocessing.GridSearchReduction(sklearn.linear_model.LogisticRegression(), prot_attr=pro_attr, constraints="DemographicParity", drop_prot_attr=False)
#inp = inprocessing.PrejudiceRemover(sensitive_attr=pro_attr)
#inp.fit(dataset_train_pred)
#dataset_train_pred = inp.predict(dataset_train_pred)
#dataset_test_pred = inp.predict(dataset_test_pred) 

#pp = postprocessing.RejectOptionClassification(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
#pp = pp.fit(dataset_train, dataset_train_pred)
#dataset_test_pred = pp.predict(dataset_test_pred)


CM = ClassificationMetric(dataset_test,
                          dataset_test_pred,
                          unprivileged_groups=unprivileged_groups,
                          privileged_groups=privileged_groups)
    
BLDM = BinaryLabelDatasetMetric(dataset_test_pred,
                                unprivileged_groups=unprivileged_groups,
                                privileged_groups=privileged_groups)

print(run_classification_metrics(CM))
print(run_binary_dataset_metrics(BLDM))

Specialized function:  compas
Optimized Preprocessing: Objective converged to 0.000000
[4.615e-01 5.489e-01 2.842e-01 7.171e-01 7.131e-01 3.581e-01 4.079e-01
 4.542e-01 4.728e-01 5.505e-01 4.361e-01 5.359e-01 6.889e-01 5.190e+02
 2.120e+02 5.340e+02 3.190e+02 2.250e+02 7.000e+01 1.740e+02 1.550e+02
 2.940e+02 1.420e+02 3.600e+02 1.640e+02 5.777e-01 5.288e-01]
[0.6394 0.6812 1.    ]
